In [ ]:
from google.colab import drive
from nltk.tokenize import word_tokenize
import nltk
# from sklearn.metrics import accuracy_score
nltk.download('punkt')
import codecs

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
import pandas as pd
from sklearn import model_selection as ms
import json
from sklearn.metrics import accuracy_score
from sklearn import model_selection
from sklearn import preprocessing
# from zeugma.embeddings import EmbeddingTransformer

In [ ]:
# from google.colab import drive
drive.mount('/content/drive')

# df1 = pd.read_csv('/content/drive/MyDrive/sar_use_data.csv')
# df2 = pd.read_csv('/content/drive/MyDrive/sar_elec_294k.csv')

Mounted at /content/drive


In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip

--2021-04-28 12:58:08--  https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 172.67.9.4, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 681808098 (650M) [application/zip]
Saving to: ‘wiki-news-300d-1M.vec.zip’

wiki-news-300d-1M.v 100%[===================>] 650.22M  15.8MB/s    in 42s     

2021-04-28 12:58:50 (15.7 MB/s) - ‘wiki-news-300d-1M.vec.zip’ saved [681808098/681808098]



In [ ]:
!unzip wiki-news-300d-1M.vec.zip

Archive:  wiki-news-300d-1M.vec.zip
  inflating: wiki-news-300d-1M.vec   


In [ ]:
df2 = df2.rename({'ELEC_embed':'embeddings'},axis = 1)

In [ ]:
df1.to_csv('/content/drive/MyDrive/sar_use_data.csv',index=False,header=True)

In [ ]:
df2.to_csv('/content/drive/MyDrive/sar_elec_294k.csv',index=False,header=True)

In [ ]:
class Model:
  def __init__(self,file,models):
    self.models = models
    df = pd.read_csv(file)
    df['tokens'] = df['text'].apply(lambda x: list(word_tokenize(x)))
    
    # if 'USE_embed' in list(df.columns):
    #   df = df.drop('Unnamed: 0',axis = 1)
    #   df = df.rename({'USE_embed':'embeddings'},axis = 1)
    # elif 'ELEC_embed' in list(df.columns):
    #   df = df.rename({'ELEC_embed':'embeddings'},axis = 1)
    
    # d = {1:'VNeg',2:'Neg',3:'Neu',4:'Pos',5:'VPos'}
    # df['label'] = df['label'].apply(lambda x: d[x])

    # skf = model_selection.StratifiedKFold(n_splits = 5)

    # for f, (t_,v_) in enumerate(skf.split(X=df,y=df.label.values)):
    #     df.loc[v_,"kfold"] = f
    self.dfs_train = [df[df.kfold != f].reset_index(drop=True) for f in range(0,5)]
    self.dfs_valid = [df[df.kfold == f].reset_index(drop=True) for f in range(0,5)]
    self.fasttext_embeddings_index = self.read_fasttext()
  


  def read_fasttext(self):
      embeddings_index = {}
      f = codecs.open('wiki-news-300d-1M.vec', encoding='utf-8')
      for line in f:
          values = line.rstrip().rsplit(' ')
          word = values[0]
          coefs = np.asarray(values[1:], dtype='float32')
          embeddings_index[word] = coefs
      f.close()
      print('found %s word vectors' % len(embeddings_index))
      return embeddings_index

  def transform(self,X):
    return np.array([
            np.mean([self.fasttext_embeddings_index[w] for w in words if w in self.fasttext_embeddings_index] 
                    or [np.zeros(300)], axis=0)
            for words in X
        ])
    
  def prepare_data(self,i):
    # mat_tr = np.matrix([x for x in  self.dfs_train[i]['embeddings'].apply(lambda x: json.loads(x))])
    # mat_ts = np.matrix([x for x in  self.dfs_valid[i]['embeddings'].apply(lambda x: json.loads(x))])
    # mat_tr = np.matrix([x for x in  self.dfs_train[i]['embeddings'].apply(lambda x: json.loads(x))])
    # mat_ts = np.matrix([x for x in  self.dfs_valid[i]['embeddings'].apply(lambda x: json.loads(x))])
    
    # x_tr= np.squeeze(np.asarray(mat_tr))
    # x_ts= np.squeeze(np.asarray(mat_ts))

    # x_tr = self.glove.transform(self.dfs_train[i]['text'])
    # x_ts = self.glove.transform(self.dfs_valid[i]['text'])
    x_tr = self.transform(self.dfs_train[i]['tokens'])
    x_ts = self.transform(self.dfs_valid[i]['tokens'])
    x_tr = preprocessing.normalize(x_tr, norm='l2')
    x_ts = preprocessing.normalize(x_ts, norm='l2')
    y_tr = self.dfs_train[i].label.values
    y_ts = self.dfs_valid[i].label.values
    return x_tr,y_tr,x_ts,y_ts
  
  def run_models(self):
    self.all = {}
    for mod in self.models:
      self.all[str(mod)] = self.pass_model(mod)
    
  def get_all(self):
    return self.all
    
  def pass_model(self,mod):
    acc = []
    for i in range(0,5):
      x_tr,y_tr,x_ts,y_ts = self.prepare_data(i)
      acc.append(self.train(mod,x_tr,y_tr,x_ts,y_ts))
    return acc

  def train(self,model,x_tr,y_tr,x_ts,y_ts):
    if model == MLPClassifier:
      mod = model(early_stopping =True)
    else:
      mod = model()
    mod.fit(x_tr,y_tr)
    return accuracy_score(y_ts,mod.predict(x_ts))


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import RidgeClassifier,LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
import numpy as np
m = Model('/content/drive/MyDrive/sarcasm data/train_sarc_final_with_folds.csv',[LogisticRegression,RidgeClassifier,GaussianNB,MLPClassifier])

found 999995 word vectors


In [ ]:
import numpy as np
m.run_models()

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


In [ ]:
x = m.get_all()

In [ ]:
print(x)

{"<class 'sklearn.linear_model._logistic.LogisticRegression'>": [0.8347619047619048, 0.8347959183673469, 0.8340136054421768, 0.8322789115646259, 0.8345068027210885], "<class 'sklearn.linear_model._ridge.RidgeClassifier'>": [0.8298469387755102, 0.8298979591836735, 0.8279251700680272, 0.8278061224489796, 0.8287925170068027], "<class 'sklearn.naive_bayes.GaussianNB'>": [0.6624829931972789, 0.6548129251700681, 0.6564455782312926, 0.6574829931972789, 0.6589285714285714], "<class 'sklearn.neural_network._multilayer_perceptron.MLPClassifier'>": [0.8593877551020408, 0.858265306122449, 0.8569727891156462, 0.8573299319727891, 0.8586054421768707]}


In [ ]:
# x = {"RandomForestClassifier": [0.7351530612244898,
#   0.7327551020408163,
#   0.7333673469387755,
#   0.7351700680272109,
#   0.7295578231292517],
#  "LogisticRegression": [0.7698299319727891,
#   0.7681292517006802,
#   0.7684353741496599,
#   0.7683163265306122,
#   0.7668027210884354],
#  "RidgeClassifier": [0.7688605442176871,
#   0.7679931972789116,
#   0.7678571428571429,
#   0.7681972789115646,
#   0.7659183673469387],
#   "GaussianNB": [0.6474659863945578, 
#                                                0.6485714285714286, 
#                                                0.6484183673469388, 
#                                                0.6487244897959183, 
#                                                0.6417006802721088],
#      "MLPClassifier": [0.7926020408163266, 
#                                                                                0.7893877551020408, 
#                                                                                0.7922959183673469, 
#                                                                                0.7929931972789116, 
#                                                                                0.7907993197278912]
#      }

y = {"LogisticRegression": [
     0.981938775510204, 
     0.9820578231292517, 
     0.9818537414965987, 
     0.9826700680272109, 
     0.981904761904762], 
     "RandomForestClassifier": [
      0.913078231292517, 
      0.9146598639455782, 
      0.9149489795918367, 
      0.9164625850340136, 
      0.9129081632653061],
     "RidgeClassifier": [
      0.9492006802721088, 
      0.9484353741496598, 
      0.9512755102040816, 
      0.950595238095238, 
      0.950952380952381], 
     "GaussianNB": [
      0.8434353741496599, 
      0.8430952380952381, 
      0.8442517006802721, 
      0.8463265306122449, 
      0.8450170068027211], 
     "MLPClassifier": [
      0.9898299319727891, 
      0.9893367346938775, 
      0.9903061224489796, 
      0.9904931972789116, 
      0.990578231292517]}




In [ ]:
x = m.get_all()

In [ ]:
x

{"<class 'sklearn.linear_model._logistic.LogisticRegression'>": [0.5717391304347826,
  0.5731811594202899,
  0.572731884057971,
  0.5696449275362319,
  0.5739057971014493],
 "<class 'sklearn.linear_model._ridge.RidgeClassifier'>": [0.5619275362318841,
  0.562427536231884,
  0.5627101449275362,
  0.5631376811594203,
  0.5658043478260869],
 "<class 'sklearn.naive_bayes.GaussianNB'>": [0.39305797101449275,
  0.3937608695652174,
  0.39555072463768115,
  0.3927608695652174,
  0.3960072463768116],
 "<class 'sklearn.neural_network._multilayer_perceptron.MLPClassifier'>": [0.6277246376811594,
  0.6286884057971015,
  0.6264420289855073,
  0.624927536231884,
  0.6272753623188406]}

In [ ]:
with open('/content/drive/MyDrive/sar_use_accuracy.txt','w') as f:
  f.write(str(y))